In [12]:
!pip install pycombo

     |████████████████████████████████| 136 kB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 213 kB 42.3 MB/s 
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Created wheel for pycombo: filename=pycombo-0.1.7-cp37-cp37m-manylinux_2_27_x86_64.whl size=98830 sha256=10d38a8559f52bcafe7fcc2f5de89460a6913875e28a730b41afb4f4e60ded8c
  Stored in directory: /root/.cache/pip/wheels/58/52/18/4c1b80cd45c091e2c1ea442729343ac984dc66b3a678e2c251
Successfully built pycombo
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.3
    Uninstalling importlib-metadata-4.11.3:
      Successfully uninstalled importlib-metadata-4.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdo

In [29]:
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans

import pycombo
import time
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

In [30]:
def propEmbed(A, alpha = 0.95):
    A = A / A.sum(axis = 1)
    n = A.shape[0]
    AI = np.linalg.inv(np.eye(n) - A * alpha)
    X = (1 - alpha) * AI
    return X


def modularity_matrix(adj):
    w_in = adj.sum(dim=0, keepdim=True)
    w_out = adj.sum(dim=1, keepdim=True)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

def modularity(Q, partition):
    return (Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum()

In [31]:
#G =  nx.karate_club_graph()
G = nx.les_miserables_graph()
#print(G.edges(data=True))
combo_comms, combo_mod = pycombo.execute(G)
A = nx.to_numpy_array(G)
X = propEmbed(A, alpha = 0.85)
c = KMeans(n_clusters = 4, n_init=100).fit(X.transpose()).labels_
adj = torch.FloatTensor(A)
Q = modularity_matrix(adj)
print(modularity(Q, c).item(), combo_mod)

0.522426426410675 0.566687983343249


In [32]:
class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNNLayer, self).__init__()
        self.weight1 = nn.Parameter(torch.randn(in_features, out_features)) # 0.5 * torch.eye(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(in_features, out_features)) # -0.5 * torch.ones(1, out_features))
        self.dropout = dropout

    def forward(self, input):
        v1 = torch.mm(input, self.weight1)
        output = v1 + self.bias
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class GNN_MLP(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(GNN_MLP, self).__init__()
        self.n_layers = 1
        self.hidden_dim = 8
        if self.n_layers > 1:
            layers = [GNNLayer(in_features, self.hidden_dim, dropout)]
        else:
            layers = [GNNLayer(in_features, out_features, dropout)]
        for _ in range(self.n_layers-2):
            layers.append(GNNLayer(self.hidden_dim, self.hidden_dim, dropout))
        if self.n_layers > 1:
            layers.append(GNNLayer(self.hidden_dim, out_features, dropout))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for i in range(self.n_layers - 1):
            x = self.layers[i](x)
            x = nn.ReLU(x)
        x = self.layers[-1](x)
        x = nn.Softmax(dim=1)(x)
        #x = 1.0 + x - x.max(dim=-1, keepdim=True).values
        #x = torch.clamp(x, 0, 1)
        #x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x

In [34]:
features = torch.FloatTensor(X.transpose())
best_best_mod = -1
for seed in range(5):
    np.random.seed(seed)
    torch.manual_seed(seed)
    t_total = time.time()
    n_comm = max(combo_comms.values()) + 2
    model = GNN_MLP(features.shape[1], n_comm)
    lr = 0.002
    n_epochs = 30000
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(n_epochs):
        t_1run = time.time()
        optimizer.zero_grad()
        out_embed = model(features)
        C = out_embed#[:, :n_comm]
        Q1 = torch.mm(C.T, Q)
        Q2 = torch.mm(Q1, C)
        loss = torch.trace(Q2)
        loss = -loss
        loss.backward()
        optimizer.step()
        if epoch == 0 or loss < best_loss:
            best_loss = loss #- torch.trace(Q)
            best_C = C.data
            best_embed = out_embed.data
            best_epoch = epoch
        if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
            #optimizer = optim.Adam(model.parameters(), lr=lr)
            print('Epoch: {:04d}'.format(epoch + 1),
                    'Modularity: {:.8f}'.format(-best_loss.item()),
                    'time: {:.4f}s'.format(time.time() - t_1run))
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    print(best_loss)
    best_best_mod = max(best_best_mod, -best_loss.item())
    #print(best_embed)
print(best_best_mod)

Epoch: 0001 Modularity: -0.00552791 time: 0.0021s
Epoch: 1501 Modularity: 0.47081259 time: 0.0006s
Epoch: 3001 Modularity: 0.47875404 time: 0.0006s
Epoch: 4501 Modularity: 0.48033568 time: 0.0006s
Epoch: 6001 Modularity: 0.48087782 time: 0.0006s
Epoch: 7501 Modularity: 0.48110086 time: 0.0006s
Epoch: 9001 Modularity: 0.48119995 time: 0.0007s
Epoch: 10501 Modularity: 0.48124561 time: 0.0007s
Epoch: 12001 Modularity: 0.48126698 time: 0.0007s
Epoch: 13501 Modularity: 0.48127708 time: 0.0006s
Epoch: 15001 Modularity: 0.48128188 time: 0.0007s
Epoch: 16501 Modularity: 0.48128411 time: 0.0007s
Epoch: 18001 Modularity: 0.48128518 time: 0.0006s
Epoch: 19501 Modularity: 0.48128569 time: 0.0007s
Epoch: 21001 Modularity: 0.48128593 time: 0.0006s
Epoch: 22501 Modularity: 0.48128608 time: 0.0016s
Epoch: 24001 Modularity: 0.48128614 time: 0.0006s
Epoch: 25501 Modularity: 0.48128617 time: 0.0006s
Epoch: 27001 Modularity: 0.48128617 time: 0.0006s
Epoch: 28501 Modularity: 0.48128617 time: 0.0006s
Epoch: